In [2]:
from collections import OrderedDict

import pandas as pd
import numpy as np

class OHLC:
    def __init__(self, filename, clean=True):
        necessary_columns = ['Date','Open','High','Low','Close','Volume']
        # Never input df data which is daily, interval should be in minutes, hours
        # nrows=10000
        self.df = pd.read_csv(filename, usecols=necessary_columns)

        # Remove unnessary columns
        # self.df.drop('6', axis=1, inplace=True)
        # self.df.drop('Unnamed: 0', axis=1, inplace=True)

        # Convert string dates to pd.Datetime
        self.df.Date = pd.DatetimeIndex(self.df.Date)

        # interval is in minues
        self.interval = None
        self.days = None
        # This is the number of data points in each day
        self.eachDayRows = None

        if clean:
            # Remove wrong data rows from df
            self.cleanDf()
            self.findInterval()

            # Assert no extra wrong rows remain now after cleanup
            assert self.eachDayRows*self.days == self.df.shape[0]

        self.df.set_index('Date', inplace=True) 
        # self.df.sort_index(inplace=True)
        
        # Remove timezone from timestamp
        self.df.index = [i.replace(tzinfo=None) for i in self.df.index]

    def toInterval(self, minutes):
        # Resamples df to the minutes input provided
        OHLCV_AGG = OrderedDict((
            ('Open', 'first'),
            ('High', 'max'),
            ('Low', 'min'),
            ('Close', 'last'),
            ('Volume', 'sum'),
        ))
        freq_minutes = pd.Series({
            "1T": 1,
            "5T": 5,
            "10T": 10,
            "15T": 15,
            "30T": 30,
            "1H": 60,
            "2H": 60*2,
            "4H": 60*4,
            "8H": 60*8,
            "1D": 60*24,
            "1W": 60*24*7,
            "1M": np.inf,
        })

        freq = freq_minutes.where(freq_minutes >= minutes).first_valid_index()
        return self.df.resample(freq, label='left', closed="left", origin='start').agg(OHLCV_AGG).dropna()

    def ge_index(self, start):
        '''
        returns index whose timeindex is greater than equal to start
        '''
        return self.df.index.searchsorted(start)
    
    def open_index(self, ind):
        '''
        returns index of first tick for the given start day
        '''
        start = self.df.index[ind]
        start = start.replace(hour=6)
        return self.ge_index(start)

    def between_time(self, start, end):
        # if string convert to pd.to_datetime("2015-03-02T09:17:00")
        return self.df[(self.df.index >= start) & (self.df.index <= end)]

    def findInterval(self):
        mp = dict()
        i = 0
        dayCnt = 0
        while i < self.df.shape[0]:
            j = i+1
            dayCnt += 1
            while j < self.df.shape[0] and self.df.Date[i].day == self.df.Date[j].day:
                interval = (self.df.Date[j] - self.df.Date[j-1]).total_seconds()
                if interval in mp:
                    mp[interval]+=1
                else:
                    mp[interval]=1
                j+=1
            i = j

        # Assert only one type of interval should exist
        assert(len(mp)) == 1

        # Dividing the interval by 60 to convert seconds to minutes
        self.interval = list(mp.keys())[0]/60
        self.days = dayCnt

    def cleanDf(self):
        def allIntervals():
            # Return a map with 
            # keys -> number of intervals in a day
            # values -> number of days with this interval
            mp = dict()
            i = 0
            while i < self.df.shape[0]:
                j = i
                while j < self.df.shape[0] and self.df.Date[i].day == self.df.Date[j].day:
                    j+=1
                if j-i in mp:
                    mp[j-i].append((i, j))
                else:
                    mp[j-i]=[(i, j)]
                i = j
            return mp
        
        mp = allIntervals()

        # Drop data which is not a most occurring number of interval in a day
        maxlen = 0
        for i in mp.values():
            maxlen = max(maxlen, len(i))
        
        # Store to be deleted indices
        rmIndx = []
        for i in mp.values():
            if len(i) < maxlen:
                for pair in i:
                    rmIndx.extend(np.arange(pair[0], pair[1]))
        
        self.df.drop(rmIndx, axis=0, inplace=True)
        self.df.index = np.arange(0, self.df.shape[0])

        # After cleanup only one type of interval should remain
        mp = allIntervals()
        assert len(mp) == 1
        self.eachDayRows = list(mp)[0]

In [3]:
class Historical:
    def __init__(self):
        self.ohlc = {}

    def get(self, symbol):
        if symbol not in self.ohlc:
            file = f'../../data/{symbol}.csv'
            if not os.path.isfile(file):
                return None
            data = OHLC(file, clean=False)
            self.ohlc.update({symbol: data})
            return data
        return self.ohlc[symbol]

historical = Historical()

In [4]:
ohlc = historical.get('ashokley')
df = ohlc.df

In [45]:
start, end = "2022-02-02 09:15:00", "2022-03-01 13:23:00"
d1 = ohlc.toInterval(60*24)

In [46]:
d1

,Open,High,Low,Close,Volume
2021-12-08 09:15:00,124.70,126.40,123.10,126.00,18442585
2021-12-09 09:15:00,127.80,128.70,125.85,127.60,16159043
2021-12-10 09:15:00,127.75,128.55,126.70,127.75,8922920
2021-12-13 09:15:00,129.20,130.25,127.65,128.15,10805892
2021-12-14 09:15:00,127.45,127.50,124.80,125.95,9475284
...,...,...,...,...,...
2022-09-27 09:15:00,154.65,155.00,149.00,151.00,14825250
2022-09-28 09:15:00,149.85,151.15,148.00,149.60,10173929
2022-09-29 09:15:00,151.15,152.50,148.75,150.00,11873933
2022-09-30 09:15:00,149.50,153.30,146.30,153.00,14572299


In [42]:
# columns: ['open_close', 'high_low', 'open_high', 'open_low']
from utils import *
open_close = get_change(df.Open, df.Close).round(2)
high_low = get_change(df.High, df.Low).round(2)
open_high = get_change(df.Open, df.High).round(2)
open_low = get_change(df.Open, df.Low).round(2)
a=pd.DataFrame({'high_low': high_low.values, 'open_close':open_close, 'open_high': open_high, 'open_low': open_low}, index=open_close.index)

In [44]:
a

,high_low,open_close,open_high,open_low
2021-12-08 09:15:00,0.69,-0.16,-0.16,0.52
2021-12-08 09:16:00,0.44,-0.20,-0.44,0.00
2021-12-08 09:17:00,0.32,0.20,-0.08,0.24
2021-12-08 09:18:00,0.28,0.16,-0.04,0.24
2021-12-08 09:19:00,0.28,-0.08,-0.24,0.04
...,...,...,...,...
2022-10-03 15:25:00,0.03,0.00,0.00,0.03
2022-10-03 15:26:00,0.17,-0.10,-0.13,0.03
2022-10-03 15:27:00,0.07,-0.03,-0.07,0.00
2022-10-03 15:28:00,0.27,0.00,-0.10,0.17


In [ ]:
a.to_json(orient='table')

In [5]:
df[-10:]

,Open,High,Low,Close,Volume
2022-10-03 15:20:00,150.55,150.60,150.25,150.30,169030
2022-10-03 15:21:00,150.30,150.50,150.30,150.50,46116
2022-10-03 15:22:00,150.50,150.60,150.50,150.60,28252
2022-10-03 15:23:00,150.60,150.75,150.55,150.70,57948
2022-10-03 15:24:00,150.75,150.90,150.70,150.90,47022
2022-10-03 15:25:00,150.90,150.90,150.85,150.90,25043
2022-10-03 15:26:00,150.90,151.10,150.85,151.05,74656
2022-10-03 15:27:00,151.05,151.15,151.05,151.10,65144
2022-10-03 15:28:00,151.00,151.15,150.75,151.00,38595
2022-10-03 15:29:00,150.95,151.00,150.70,150.70,27932
